# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas e linhas do DataFrame
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None) 

# Base de dados

In [2]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"D:\OneDrive\Trabalho\ESA 2025\Dashboard_ESA_2025\base_dados\Cel Manfrini.csv"

caminho_2 = r"D:\OneDrive\Trabalho\ESA 2025\Dashboard_ESA_2025\base_dados\Cel Manfrini(1).csv"
#
#caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"
#
#caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"
#
#caminho_5 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(4).csv"
#
#caminho_6 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(5).csv"



# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
#df3 = pd.read_csv(caminho_3)
#df4 = pd.read_csv(caminho_4)
#df5 = pd.read_csv(caminho_5)
#df6 = pd.read_csv(caminho_6)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Inserindo _ nos titulos das colunas
df.columns = df.columns.str.replace(' ', '_')

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)

# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [3]:
df_saldos = df[['ANO','UG', 'CHAVE' ,'NOME_CREDOR', 'FAV', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [4]:
df_saldos.to_csv("saldos_esa_2025.csv", index=False)

In [5]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 5.372.839,54
O total de saldo a liquidar pela ESA até o momento foi: R$ 9.692.382,63
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 806.546,62
O total de saldo a pagar pela ESA até o momento foi: R$ 10.498.929,25


In [7]:
df_saldos.columns

Index(['ANO', 'UG', 'CHAVE', 'NOME_CREDOR', 'FAV', 'DATA', 'DIAS', 'ND',
       'NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI', 'TIPO', 'NOME_UGR',
       'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO'],
      dtype='object')

In [7]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_white')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [8]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [9]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [13]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [14]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [15]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
61,2023NE000033,3236.21,752,PASSAGENS PARA O PAIS,33903301
237,2023NE001678,4794.00,493,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
199,2024NE000011,9273.13,398,PASSAGENS PARA O PAIS,33903301
200,2024NE000124,3268.98,364,PASSAGENS PARA O PAIS,33903301
217,2024NE000167,11216.60,363,SERVICOS DOMESTICOS,33903946
221,2024NE000238,836.08,355,SERVICOS DE COMUNICACAO EM GERAL,33903947
153,2024NE000269,0.56,346,MATERIAL P/ MANUTENCAO DE VEICULOS,33903039
218,2024NE000346,3988.00,326,SERVICOS DOMESTICOS,33903946
64,2024NE000354,30215.64,322,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
13,2024NE000363,20812.48,320,SERVICOS DE COMUNICACAO EM GERAL,33903947


In [16]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [17]:
# Qtde de empenhos realizados até o momento
df.shape[0]

789

# Requisição em API para consulta a CNPJ

In [18]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/03126110000192"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '04/05/1999',
 'atividade_principal': [{'code': '49.30-2-04',
                          'text': 'Transporte rodoviário de mudanças'}],
 'atividades_secundarias': [{'code': '49.30-2-01',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'municipal'},
                            {'code': '49.30-2-02',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'intermunicipal, interestadual e '
                                     'internacional'},
                            {'code': '52.11-7-02', 'text': 'Guarda-móveis'},
                            {'code': '68.10-2-02',
                             'text': 'Aluguel de imóveis próprios'}],
 'bairro': 'ZONA INDUSTRIAL (GUARA)',
 'billing': {'database': True, 'free

In [19]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [20]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [21]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [22]:
dicionario['telefone']

'(41) 3316-7700'

In [23]:
dicionario['email']

'fiscal@positivo.com.br'

In [24]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'RUA JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [25]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['FAV'] != '160129']

df_cnpj['FAV'].unique()

array(['39750831000116', '13081240000100', '05872523000141',
       '56062746000184', '17281106000103', '34028316001509',
       '06981180000116', '65295172000185', '02421421000111',
       '20531237000106', '01255291000121', '29423785000197',
       '70999289000180', '68533967000172', '51149498000190',
       '21331404000138', '19791896011307', '20702528000110',
       '10653955000100', '20942789000107', '18563967000147',
       '71001465000105', '33180160000119', '47348831000112',
       '37748531000177', '09294294000186', '47078180000198',
       '20776492000119', '03958259626', '08343486838', '04638751172',
       '24782217000169', '02338743000100', '00295234000103',
       '18707234000139', '14807420000199', '33658130000175',
       '70797366334', '08528728609', '01368832652', '39644514000115',
       '15653078794', '01737297388', '06971819608', '12098511647',
       '14441938721', '04160798730', '45654847813', '01052392245',
       '05573004794', '10220358605', '32413411860', '04

In [26]:
df_cnpj[df_cnpj['FAV'] == '160129']

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [27]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente.head(2)

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
3,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,I3DA,CNT,LICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2025NE000002,2025,2025-01-15,160129000012025NE000002,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,40,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,36054.83,0.00,36054.83,0.0
4,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,I3DA,CNT,LICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2025NE000003,2025,2025-01-15,160129000012025NE000003,13081240000100,SIGMA-SERVICOS TERCEIRIZADOS LTDA,PR,ANO,40,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,0.00,41497.34,41497.34,0.0


# Pesquisas no DataFrame

In [28]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos inscritos em RPNP

In [29]:
# Criando um Dataframe para filtrar os empenhos em RPNP
df_restos = df_teste[df_teste['ANO'].isin(['2022', '2023'])]
df_restos = df_restos[df_restos['A_LIQUIDAR'] !=  0]

### Restos a pagar ND 30 e 52

In [30]:
df_restos_material = df_restos[df_restos['ND'].isin(['339030', '449052' ])]
df_restos_material = df_restos_material.sort_values(by='A_LIQUIDAR', ascending=False)
df_restos_material.head()
df_restos_material.to_excel('restos_a_pagar_2024.xlsx')

In [31]:
cols = ['UG', 'NOME_ACAO', 'ANO', 'NOME_CREDOR',
'DATA', 'DIAS', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'PI', 
'NOME_PI', 'TIPO', 'TIPO_NE', 'NOME_UGR', 'A_LIQUIDAR', 
'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_filtro = df_teste.loc[:,cols]
df_filtro = df_filtro[df_filtro['A_LIQUIDAR'] != 0]
df_filtro.sort_values(by='DIAS', ascending=False).head(2)

,UG,NOME_ACAO,ANO,NOME_CREDOR,DATA,DIAS,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,TIPO,TIPO_NE,NOME_UGR,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
61,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,FACTO TURISMO LTDA,2023-02-03,752,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000033,C4ENMILCAPE,CAPACITACAO DE PESSOAL,RPNP,GLOBAL,DECEX/GESTOR,3236.21,0.0,3236.21,0.0
237,167129,ADMINISTRACAO DA UNIDADE,2023,COPYUAI LOCACAO DE EQUIPAMENTOS LTDA,2023-10-20,493,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2023NE001678,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,RPNP,GLOBAL,DECEX - GESTOR,4794.00,0.0,4794.00,0.0


### Filtros Diversos

In [32]:
df_liquidar = df_teste[df_teste['A_LIQUIDAR'] != 0].sort_values(by= 'DIAS', ascending= False).reset_index(drop=True)

In [33]:
cols = ['UG','NOME_ACAO', 'NOME_UGR', 'PI',
       'NOME_PI', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'ANO', 'DATA',
       'FAV', 'NOME_CREDOR', 'UF_FAV', 'TIPO', 'DIAS', 'UF_UG',
       'MOD_LIC', 'TIPO_NE', 'RP', 'NOME_RP', 'UO', 'NOME_UO', 'A_LIQUIDAR',
       'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_liquidar.loc[:, cols].head()

,UG,NOME_ACAO,NOME_UGR,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,MOD_LIC,TIPO_NE,RP,NOME_RP,UO,NOME_UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000033,2023,2023-02-03,14807420000199,FACTO TURISMO LTDA,SP,RPNP,752,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,3236.21,0.00,3236.21,0.00
1,167129,ADMINISTRACAO DA UNIDADE,DECEX - GESTOR,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2023NE001678,2023,2023-10-20,05691235000190,COPYUAI LOCACAO DE EQUIPAMENTOS LTDA,MG,RPNP,493,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52921,FUNDO DO EXERCITO,4794.00,0.00,4794.00,0.00
2,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2024NE000011,2024,2024-01-23,14807420000199,FACTO TURISMO LTDA,SP,RPNP,398,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,9273.13,5828.30,15101.43,0.00
3,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2024NE000124,2024,2024-02-26,21331404000138,ORLEANS VIAGENS E TURISMO LTDA,SP,RPNP,364,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,3268.98,65.78,3334.76,867.41
4,160129,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2024NE000167,2024,2024-02-27,15272846000158,PS SERVICOS DE LAVANDERIA LTDA,RN,RPNP,363,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,11216.60,0.00,11216.60,0.00


In [34]:
df_liquidar.to_excel('empenhos_2024_liquidar.xlsx')

In [35]:
df_empenhos_mais_60_dias = df_teste[(df_teste['DIAS'] >= 60)& (df_teste['A_LIQUIDAR'] != 0)].sort_values(by="A_LIQUIDAR", ascending=False)

df_empenhos_mais_60_dias.to_excel('empenhos_mais_60_dias.xlsx')

In [36]:
df_ensino = df_teste[df_teste['AÇÃO'] == '8965']

In [37]:
total_pago_ensino = df_ensino['PAGO'].sum().round(2)
total_a_liquidar_ensino = df_ensino['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar_ensino = df_ensino['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar_ensino = df_ensino['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago_ensino, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar_ensino, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar_ensino, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar_ensino, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 308.118,80
O total de saldo a liquidar pela ESA até o momento foi: R$ 1.331.228,13
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 76.391,62
O total de saldo a pagar pela ESA até o momento foi: R$ 1.407.619,75


In [38]:
df_ensino.sort_values(by='A_LIQUIDAR', ascending=False).head(5)

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
207,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903309,TRANSPORTE DE SERVIDORES,2024NE000810,2024,2024-07-03,160129000012024NE000810,05824788000174,EMPRESA DE TRANSPORTES KOPERECK LTDA,RS,RPNP,236,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,484382.76,0.00,484382.76,0.0
210,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903309,TRANSPORTE DE SERVIDORES,2024NE001947,2024,2024-11-13,160129000012024NE001947,05824788000174,EMPRESA DE TRANSPORTES KOPERECK LTDA,RS,RPNP,103,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,188028.46,0.00,188028.46,0.0
216,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903922,"EXPOSICOES, CONGRESSOS E CONFERENCIAS",2024NE002594,2024,2024-12-20,160129000012024NE002594,07851262000109,MV EVENTOS ARTISTICOS E ESPORTIVOS LTDA,MG,RPNP,66,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,157576.00,0.00,157576.00,0.0
219,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2024NE002564,2024,2024-12-06,160129000012024NE002564,15272846000158,PS SERVICOS DE LAVANDERIA LTDA,RN,RPNP,80,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,88587.62,12567.28,101154.90,0.0
206,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,232082,C4ENMILCAPE,C4EN,MIL,CAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903307,DESPESAS COM EXCESSO DE BAGAGEM,2024NE001049,2024,2024-09-19,160129000012024NE001049,05824788000174,EMPRESA DE TRANSPORTES KOPERECK LTDA,RS,RPNP,158,MG,3000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,COMANDO DO EXERCITO,52121,55995.30,0.00,55995.30,0.0


In [39]:
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df_ensino.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
#df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]

df_aux.to_excel('ensino_recursos_2024.xlsx')

In [40]:
df_transporte = df_teste[df_teste['ND'] == '339033']

total_pago_ensino = df_transporte['PAGO'].sum().round(2)
total_a_liquidar_ensino = df_transporte['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar_ensino = df_transporte['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar_ensino = df_transporte['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago_ensino, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar_ensino, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar_ensino, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar_ensino, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 82.546,59
O total de saldo a liquidar pela ESA até o momento foi: R$ 2.264.838,99
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 31.112,95
O total de saldo a pagar pela ESA até o momento foi: R$ 2.295.951,94


In [41]:
df_transporte.to_csv('nd33_2024.csv')

In [42]:
df_discricionarias = df_teste[(df_teste['AÇÃO'] == '2000') & (df_teste['A_LIQUIDAR'] != 0)]

In [43]:
df_teste[(df_teste['AÇÃO'] == '212O') & (df_teste['A_LIQUIDAR'] != 0)]

,UG,SIGLA,CGCFEX,RM,COMANDO,CIDADE,UF,PROGRAMA,AÇÃO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,RADICAL_4_PI,MEIO_3_PI,SUFIXO_4_PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,NOME_UO,UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
65,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D1PEINDCL4T,D1PE,IND,CL4T,IND - INDENIZACOES,339093,INDENIZACOES E RESTITUICOES,33909319,INDENIZACAO DE TRANSPORTE - PESSOAL MILITAR,2025NE000080,2025,2025-02-21,160129000012025NE000080,70797366334,MARGOLLIN MORAIS DA SILVA,MG,ANO,3,MG,1000000000,NAO SE APLICA,ORDINARIO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,0,COMANDO DO EXERCITO,52121,1630.20,0.0,1630.20,0.00
66,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D1PEINDCL4T,D1PE,IND,CL4T,IND - INDENIZACOES,339093,INDENIZACOES E RESTITUICOES,33909319,INDENIZACAO DE TRANSPORTE - PESSOAL MILITAR,2025NE000081,2025,2025-02-21,160129000012025NE000081,08528728609,HUDSON MARCOS PACHECO DE SOUZA,SP,ANO,3,MG,1000000000,NAO SE APLICA,ORDINARIO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,0,COMANDO DO EXERCITO,52121,1482.00,0.0,1482.00,0.00
67,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D1PEINDCL4T,D1PE,IND,CL4T,IND - INDENIZACOES,339093,INDENIZACOES E RESTITUICOES,33909319,INDENIZACAO DE TRANSPORTE - PESSOAL MILITAR,2025NE000082,2025,2025-02-21,160129000012025NE000082,01368832652,CASSIO DE FREITAS NASCIMENTO,MG,ANO,3,MG,1000000000,NAO SE APLICA,ORDINARIO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,0,COMANDO DO EXERCITO,52121,1482.00,0.0,1482.00,0.00
69,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000717,2024,2024-06-20,160129000012024NE000717,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,RPNP,249,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,19850.00,0.0,19850.00,0.00
70,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000718,2024,2024-06-20,160129000012024NE000718,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,RPNP,249,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,34684.00,0.0,34684.00,0.00
71,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000719,2024,2024-06-20,160129000012024NE000719,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,RPNP,249,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,COMANDO DO EXERCITO,52121,50049.00,0.0,50049.00,0.00
72,160129,ESA,4º CGCFEx,4ª RM,CML,Três Corações,MG,6112,212O,MOVIMENTACAO DE MILITARES,160505,DGP,232007,D6DAATSLOCO,D6DA,ATS,LOCO,ATS - ATIVIDADES SETORIAIS,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2024NE000742,2024,2024-06-25,160129000012024NE000742,39644514000115,MULTI PRIME MUDANCAS E TRANSPORTES ESPECIAIS LTDA,GO,RPNP,244,MG,1000000000,PREGAO,ESTIMATIVO,1,PRIMARIO OBRIGATORIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESC